In [1]:
import pandas as pd
import numpy as np
from nltk import word_tokenize, sent_tokenize

In [2]:
training_data = pd.read_csv('train.csv',delimiter=',')
training_data

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...,...
10975,569934458364813313,neutral,American,NaN,Cottopanama85,NaN,0,@AmericanAir followback,NaN,2015-02-23 10:58:58 -0800,"ohio,panama",NaN
10976,568564006329434113,positive,United,NaN,PaulBEsteves,NaN,0,@united thanks for the help. Wish the phone re...,NaN,2015-02-19 16:13:17 -0800,Brooklyn,Eastern Time (US & Canada)
10977,569643648910028801,negative,US Airways,NaN,runfixsteve,NaN,0,@usairways the. Worst. Ever. #dca #customerser...,NaN,2015-02-22 15:43:24 -0800,"St. Augustine, Florida",NaN
10978,568864981917110272,negative,US Airways,NaN,CLChicosky,NaN,0,@nrhodes85: look! Another apology. DO NOT FLY ...,NaN,2015-02-20 12:09:15 -0800,NaN,NaN


In [3]:
training_data["text"][0]

'@SouthwestAir I am scheduled for the morning, 2 days after the fact, yes..not sure why my evening flight was the only one Cancelled Flightled'

In [4]:
testing_data = pd.read_csv('test.csv',delimiter=',')
testing_data

,tweet_id,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,569682010270101504,American,NaN,zsalim03,NaN,0,@AmericanAir In car gng to DFW. Pulled over 1h...,NaN,2015-02-22 18:15:50 -0800,Texas,Central Time (US & Canada)
1,569608307184242688,American,NaN,sa_craig,NaN,0,"@AmericanAir after all, the plane didn’t land ...",NaN,2015-02-22 13:22:57 -0800,"College Station, TX",Central Time (US & Canada)
2,567879304593408001,Southwest,NaN,DanaChristos,NaN,1,@SouthwestAir can't believe how many paying cu...,NaN,2015-02-17 18:52:31 -0800,CT,Eastern Time (US & Canada)
3,569757651539660801,US Airways,NaN,rossj987,NaN,0,@USAirways I can legitimately say that I would...,NaN,2015-02-22 23:16:24 -0800,"Washington, D.C.",Eastern Time (US & Canada)
4,569900705852608513,American,NaN,tranpham18,NaN,0,@AmericanAir still no response from AA. great ...,NaN,2015-02-23 08:44:51 -0800,New York City,Eastern Time (US & Canada)
...,...,...,...,...,...,...,...,...,...,...,...
3655,570304244001193984,US Airways,NaN,Anthony_Scerri,NaN,0,@USAirways Been stuck for 40+ minutes due to l...,NaN,2015-02-24 11:28:22 -0800,"Astoria, NY",Quito
3656,567847737061941249,US Airways,NaN,mttdprkr,NaN,0,@USAirways 4 hours... 4 hours... FOUR HOURS. ...,NaN,2015-02-17 16:47:05 -0800,"Vancouver, WA",Pacific Time (US & Canada)
3657,567823564167192576,Virgin America,NaN,miaerolinea,NaN,1,Nice RT @VirginAmerica: The man of steel might...,NaN,2015-02-17 15:11:02 -0800,Worldwide,Caracas
3658,570273819287531520,American,NaN,GoldensPleasure,NaN,0,@AmericanAir Aww Thanks AA..DFW was on GMA up ...,NaN,2015-02-24 09:27:28 -0800,East Coast CT.,Central Time (US & Canada)


In [5]:
documents=[]
for i in range(len(training_data)):
    documents.append((word_tokenize(training_data["text"][i]),training_data["airline_sentiment"][i]))
documents[0:5]

[(['@',
   'SouthwestAir',
   'I',
   'am',
   'scheduled',
   'for',
   'the',
   'morning',
   ',',
   '2',
   'days',
   'after',
   'the',
   'fact',
   ',',
   'yes',
   '..',
   'not',
   'sure',
   'why',
   'my',
   'evening',
   'flight',
   'was',
   'the',
   'only',
   'one',
   'Cancelled',
   'Flightled'],
  'negative'),
 (['@',
   'SouthwestAir',
   'seeing',
   'your',
   'workers',
   'time',
   'in',
   'and',
   'time',
   'out',
   'going',
   'above',
   'and',
   'beyond',
   'is',
   'why',
   'I',
   'love',
   'flying',
   'with',
   'you',
   'guys',
   '.',
   'Thank',
   'you',
   '!'],
  'positive'),
 (['@',
   'united',
   'Flew',
   'ORD',
   'to',
   'Miami',
   'and',
   'back',
   'and',
   'had',
   'great',
   'crew',
   ',',
   'service',
   'on',
   'both',
   'legs',
   '.',
   'THANKS'],
  'positive'),
 (['@',
   'SouthwestAir',
   '@',
   'dultch97',
   'that',
   "'s",
   'horse',
   'radish',
   '😤🐴'],
  'negative'),
 (['@',
   'united',
   's

In [27]:
test_documents=[]
for i in range(len(testing_data)):
    test_documents.append(word_tokenize(testing_data["text"][i]))
test_documents[0:5]

[['@',
  'AmericanAir',
  'In',
  'car',
  'gng',
  'to',
  'DFW',
  '.',
  'Pulled',
  'over',
  '1hr',
  'ago',
  '-',
  'very',
  'icy',
  'roads',
  '.',
  'On-hold',
  'with',
  'AA',
  'since',
  '1hr',
  '.',
  'Ca',
  "n't",
  'reach',
  'arpt',
  'for',
  'AA2450',
  '.',
  'Wat',
  '2',
  'do',
  '?'],
 ['@',
  'AmericanAir',
  'after',
  'all',
  ',',
  'the',
  'plane',
  'didn',
  '’',
  't',
  'land',
  'in',
  'identical',
  'or',
  'worse',
  ')',
  'conditions',
  'at',
  'GRK',
  'according',
  'to',
  'METARs',
  '.'],
 ['@',
  'SouthwestAir',
  'ca',
  "n't",
  'believe',
  'how',
  'many',
  'paying',
  'customers',
  'you',
  'left',
  'high',
  'and',
  'dry',
  'with',
  'no',
  'reason',
  'for',
  'flight',
  'Cancelled',
  'Flightlations',
  'Monday',
  'out',
  'of',
  'BDL',
  '!',
  'Wow',
  '.'],
 ['@',
  'USAirways',
  'I',
  'can',
  'legitimately',
  'say',
  'that',
  'I',
  'would',
  'have',
  'rather',
  'driven',
  'cross',
  'country',
  'than',


In [6]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [7]:
from nltk.corpus import wordnet
from nltk import pos_tag
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [8]:
from nltk.corpus import stopwords
import string
stops = set(stopwords.words('english'))
punctuations = list(string.punctuation)
stops.update(punctuations)
stops

{'!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'need

In [14]:
def clean_review(words):
    output_words = []
    for w in words:
        if w.lower().isalpha()==True and w.lower() not in stops:
            pos = pos_tag([w])
            clean_word = lemmatizer.lemmatize(w,pos=get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
    return output_words

In [29]:
documents = [(clean_review(doc),category)for doc,category in documents]
test_documents = [(clean_review(doc))for doc in test_documents]

In [30]:
documents[0]

(['southwestair',
  'schedule',
  'morning',
  'day',
  'fact',
  'yes',
  'sure',
  'even',
  'flight',
  'one',
  'cancel',
  'flightled'],
 'negative')

In [31]:
test_documents[0]

['americanair',
 'car',
 'gng',
 'dfw',
 'pulled',
 'ago',
 'icy',
 'road',
 'aa',
 'since',
 'ca',
 'reach',
 'arpt',
 'wat']

In [17]:
all_words=[]
for doc,category in documents:
    all_words += doc
all_words

['southwestair',
 'schedule',
 'morning',
 'day',
 'fact',
 'yes',
 'sure',
 'even',
 'flight',
 'one',
 'cancel',
 'flightled',
 'southwestair',
 'see',
 'worker',
 'time',
 'time',
 'go',
 'beyond',
 'love',
 'fly',
 'guy',
 'thank',
 'united',
 'flew',
 'ord',
 'miami',
 'back',
 'great',
 'crew',
 'service',
 'leg',
 'thanks',
 'southwestair',
 'horse',
 'radish',
 'united',
 'flight',
 'ord',
 'delayed',
 'air',
 'force',
 'one',
 'last',
 'flight',
 'sbn',
 'min',
 'land',
 'united',
 'load',
 'u',
 'fly',
 'sardine',
 'knew',
 'pilot',
 'hour',
 'late',
 'flight',
 'incompetent',
 'beyond',
 'belief',
 'jetblue',
 'stock',
 'response',
 'delay',
 'frustrate',
 'poor',
 'cust',
 'serv',
 'amp',
 'told',
 'ppl',
 'wait',
 'amp',
 'come',
 'back',
 'jetblue',
 'nice',
 'hop',
 'rack',
 'enough',
 'mile',
 'take',
 'trip',
 'seattle',
 'enjoy',
 'perfect',
 'latte',
 'city',
 'coffee',
 'united',
 'frankly',
 'bad',
 'customer',
 'service',
 'ever',
 'problem',
 'happen',
 'deal',
 

In [18]:
import nltk
freq = nltk.FreqDist(all_words)
common = freq.most_common(2000)
features = [i[0] for i in common]

In [19]:
features

['flight',
 'united',
 'usairways',
 'americanair',
 'southwestair',
 'jetblue',
 'get',
 'http',
 'hour',
 'thanks',
 'cancel',
 'u',
 'service',
 'help',
 'customer',
 'time',
 'call',
 'bag',
 'wait',
 'go',
 'plane',
 'make',
 'fly',
 'need',
 'amp',
 'hold',
 'thank',
 'would',
 'still',
 'try',
 'one',
 'airline',
 'gate',
 'day',
 'please',
 'back',
 'ca',
 'take',
 'delayed',
 'book',
 'say',
 'virginamerica',
 'flightled',
 'bad',
 'seat',
 'delay',
 'like',
 'change',
 'agent',
 'phone',
 'late',
 'miss',
 'today',
 'guy',
 'know',
 'ticket',
 'work',
 'airport',
 'could',
 'give',
 'minute',
 'way',
 'great',
 'lose',
 'well',
 'check',
 'never',
 'travel',
 'even',
 'min',
 'see',
 'problem',
 'want',
 'really',
 'home',
 'hr',
 'look',
 'tomorrow',
 'weather',
 'dm',
 'last',
 'good',
 'another',
 'use',
 'people',
 'love',
 'email',
 'issue',
 'sit',
 'new',
 'let',
 'first',
 'told',
 'yes',
 'much',
 'luggage',
 'ever',
 'due',
 'number',
 'next',
 'someone',
 'trip',
 

In [20]:
def get_feature_dic(words):
    feature_dict={}
    words_set = set(words)
    for w in features:
        feature_dict[w] = w in words_set
    return feature_dict

In [22]:
temp = get_feature_dic(documents[0][0])
temp

{'flight': True,
 'united': False,
 'usairways': False,
 'americanair': False,
 'southwestair': True,
 'jetblue': False,
 'get': False,
 'http': False,
 'hour': False,
 'thanks': False,
 'cancel': True,
 'u': False,
 'service': False,
 'help': False,
 'customer': False,
 'time': False,
 'call': False,
 'bag': False,
 'wait': False,
 'go': False,
 'plane': False,
 'make': False,
 'fly': False,
 'need': False,
 'amp': False,
 'hold': False,
 'thank': False,
 'would': False,
 'still': False,
 'try': False,
 'one': True,
 'airline': False,
 'gate': False,
 'day': True,
 'please': False,
 'back': False,
 'ca': False,
 'take': False,
 'delayed': False,
 'book': False,
 'say': False,
 'virginamerica': False,
 'flightled': True,
 'bad': False,
 'seat': False,
 'delay': False,
 'like': False,
 'change': False,
 'agent': False,
 'phone': False,
 'late': False,
 'miss': False,
 'today': False,
 'guy': False,
 'know': False,
 'ticket': False,
 'work': False,
 'airport': False,
 'could': False,
 'g

In [34]:
training_data = [ (get_feature_dic(doc),category)for doc,category in documents]
testing_data = [ (get_feature_dic(doc))for doc in test_documents]

In [25]:
training_data[0]

({'flight': True,
  'united': False,
  'usairways': False,
  'americanair': False,
  'southwestair': True,
  'jetblue': False,
  'get': False,
  'http': False,
  'hour': False,
  'thanks': False,
  'cancel': True,
  'u': False,
  'service': False,
  'help': False,
  'customer': False,
  'time': False,
  'call': False,
  'bag': False,
  'wait': False,
  'go': False,
  'plane': False,
  'make': False,
  'fly': False,
  'need': False,
  'amp': False,
  'hold': False,
  'thank': False,
  'would': False,
  'still': False,
  'try': False,
  'one': True,
  'airline': False,
  'gate': False,
  'day': True,
  'please': False,
  'back': False,
  'ca': False,
  'take': False,
  'delayed': False,
  'book': False,
  'say': False,
  'virginamerica': False,
  'flightled': True,
  'bad': False,
  'seat': False,
  'delay': False,
  'like': False,
  'change': False,
  'agent': False,
  'phone': False,
  'late': False,
  'miss': False,
  'today': False,
  'guy': False,
  'know': False,
  'ticket': False,

In [35]:
testing_data[0]

{'flight': False,
 'united': False,
 'usairways': False,
 'americanair': True,
 'southwestair': False,
 'jetblue': False,
 'get': False,
 'http': False,
 'hour': False,
 'thanks': False,
 'cancel': False,
 'u': False,
 'service': False,
 'help': False,
 'customer': False,
 'time': False,
 'call': False,
 'bag': False,
 'wait': False,
 'go': False,
 'plane': False,
 'make': False,
 'fly': False,
 'need': False,
 'amp': False,
 'hold': False,
 'thank': False,
 'would': False,
 'still': False,
 'try': False,
 'one': False,
 'airline': False,
 'gate': False,
 'day': False,
 'please': False,
 'back': False,
 'ca': True,
 'take': False,
 'delayed': False,
 'book': False,
 'say': False,
 'virginamerica': False,
 'flightled': False,
 'bad': False,
 'seat': False,
 'delay': False,
 'like': False,
 'change': False,
 'agent': False,
 'phone': False,
 'late': False,
 'miss': False,
 'today': False,
 'guy': False,
 'know': False,
 'ticket': False,
 'work': False,
 'airport': False,
 'could': False,

In [26]:
from nltk import NaiveBayesClassifier

In [36]:
classifier = NaiveBayesClassifier.train(training_data)

In [37]:
classifier.show_most_informative_features(15)

Most Informative Features
                   kudos = True           positi : negati =     51.9 : 1.0
               fantastic = True           positi : negati =     34.2 : 1.0
                favorite = True           positi : negati =     34.2 : 1.0
             outstanding = True           positi : negati =     29.1 : 1.0
                  dragon = True           neutra : negati =     28.5 : 1.0
                 helpful = True           positi : neutra =     26.3 : 1.0
                   flyfi = True           positi : negati =     24.1 : 1.0
                   thank = True           positi : negati =     22.6 : 1.0
                 awesome = True           positi : negati =     22.6 : 1.0
                    hold = True           negati : positi =     22.3 : 1.0
               beautiful = True           positi : negati =     21.5 : 1.0
                      hr = True           negati : positi =     21.4 : 1.0
                   daily = True           neutra : negati =     20.6 : 1.0

In [38]:
naive = classifier.classify_many(testing_data)
naive

['negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'neutral',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'negative',
 'neutral',
 'negative',
 'positive',
 'neutral',
 'negative',
 'negative',
 'positive',
 'neutral',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'neutral',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'neutral',
 'neutral',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'positive',
 'positive',
 'negative',
 'neutral',
 'negative',
 'negative',
 'negative',
 'neutral',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative

In [48]:
data = pd.DataFrame(naive)
data.to_csv('predictions.csv',index=False,header=False)

In [51]:
from nltk.classify.scikitlearn import SklearnClassifier

In [49]:
from sklearn.ensemble import RandomForestClassifier

In [52]:
rfc = RandomForestClassifier()
classifier_sklearn = SklearnClassifier(rfc)
classifier_sklearn.train(training_data)

<SklearnClassifier(RandomForestClassifier())>

In [53]:
random_forest = classifier_sklearn.classify_many(testing_data)

In [56]:
data = pd.DataFrame(random_forest)
data.to_csv('final_predictions.csv',index=False,header=False)